In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
DATA_FOLDER = 'data'

In [3]:
dinner_nutrients_df = pd.read_csv('dinner_nutrient.csv', index_col=0)
#dinner_ingredients_df = pd.read_csv('dinner_ingredient.csv', index_col = "id")

## What we need to do

**Cleaning:**
1. Make quantities of nutrients into integers (while respecting units)
2. Create vegetarian dataset by removing all dishes containing meat or fish-related ingredients (list of words found in the corresponding text documents under /data). If a meat/fish is in ingredients, delete the corresponding recipe from the nutrient dataset.

**Analysis:**
1. Plot histograms of the quantity of a given nutrient thoughout the vegetarian recipes
2. Calculate median and mode (= most common value)
3. Compare these with the FDA recommendation. NOTE this needs to be per serving - verify
4. Calculate median and mode from omnivorous (original) dataset. For large differences with the vegetarian data, also view histogram. Compare together and with FDA.
5. For vegetarian histograms with large discrepencies (some recipes with high levels of nutrient, others with low levels), cross-reference ingredients to see what may be responsible for providing the nutrient.

**Visualization of results:**
1. It would be good to plot the FDA recommendation on the histogram, also median and mode

### Lets explore the data and do some cleaning

In [5]:
dinner_nutrients_df.head(20)

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smoth...,20.1g,Total Fat,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/222607/smoth...,8.0g,Saturated Fat,Smothered Chicken Breasts
2,https://www.allrecipes.com/recipe/222607/smoth...,124mg,Cholesterol,Smothered Chicken Breasts
3,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
4,https://www.allrecipes.com/recipe/222607/smoth...,377mg,Potassium,Smothered Chicken Breasts
5,https://www.allrecipes.com/recipe/222607/smoth...,16.9g,Total Carbohydrates,Smothered Chicken Breasts
6,https://www.allrecipes.com/recipe/222607/smoth...,0.5g,Dietary Fiber,Smothered Chicken Breasts
7,https://www.allrecipes.com/recipe/222607/smoth...,44g,Protein,Smothered Chicken Breasts
8,https://www.allrecipes.com/recipe/222607/smoth...,15g,Sugars,Smothered Chicken Breasts
9,https://www.allrecipes.com/recipe/222607/smoth...,29IU,Vitamin A,Smothered Chicken Breasts


In [7]:
sodium = dinner_nutrients_df.where(dinner_nutrients_df['nutrient'] == "Sodium")

In [8]:
sodium = sodium.dropna()
sodium = sodium.reset_index(drop=True)
sodium.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/15679/asian-...,711mg,Sodium,Asian Beef with Snow Peas
2,https://www.allrecipes.com/recipe/23847/pasta-...,350mg,Sodium,Pasta Pomodoro
3,https://www.allrecipes.com/recipe/50435/fry-br...,2255mg,Sodium,Fry Bread Tacos II
4,https://www.allrecipes.com/recipe/140829/pork-...,356mg,Sodium,Pork Marsala


The unit is the same for each value of a given nutrient, therefore we can drop this value with impunity (verification using Sodium since this value can vary greatly)

Vitamin A will be completely ignored since it is given in IU. This unit will no longer be valid from 2021. In addition, it cannot be converted into micrograms since this depends on its origin: retinol (pre-formed vitamin A) or beta-carotene (a precursor).

In [9]:
dinner_nutrients_df = dinner_nutrients_df[dinner_nutrients_df.nutrient != "Vitamin A"]

In [10]:
dinner_nutrients_df.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smoth...,20.1g,Total Fat,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/222607/smoth...,8.0g,Saturated Fat,Smothered Chicken Breasts
2,https://www.allrecipes.com/recipe/222607/smoth...,124mg,Cholesterol,Smothered Chicken Breasts
3,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
4,https://www.allrecipes.com/recipe/222607/smoth...,377mg,Potassium,Smothered Chicken Breasts


In [11]:
#Remove the letters m, c, g, I, U from the string nutrient and convert it to a float

dinner_nutrients_df.amount = dinner_nutrients_df.amount.apply(lambda x: re.sub('[mcgIU]','', x))
dinner_nutrients_df.amount = dinner_nutrients_df.amount.replace(to_replace='< 1', value='0')
dinner_nutrients_df.amount = dinner_nutrients_df.amount.astype(float)
dinner_nutrients_df.dtypes

URL          object
amount      float64
nutrient     object
recipe       object
dtype: object

**Now our data is clean, we can create a vegetarian dataframe: **

In [12]:
meat_df = pd.read_csv('data/exceptionLists/meats', header=None)
meat_df.head()
#dinner_nutrient_vege_df = 

,0
0,beef
1,chicken
2,pork
3,filet
4,fillet
